# Prepare and understand data for modeling

모든 데이터는 보통 지저분하고 데이터가 의도한 것에 대한 충분한 신뢰성을 가지고 있지 않다.

데이터는 중복 데이터나 관찰되지 않은 값 (결측치), 아웃라이어, 존재하지 않는 주소, 잘못된 전화번호 또는 지역코드, 올바르지 않은 지역좌표, 잘못된 데이터나 레이블, 대소문자 구분, 공백 관련 문제를 가지고 있다.

데이터과학자나 데이터엔지니어는 통계 모델 또는 머신러닝 모델을 빌드하기 위해 이러한 데이터를 깨끗하게 만들어야 한다.

데이터는 앞에서 말한 문제점들이 없을 경우 기술적으로 깨끗하다고 말 할 수 있다.
그러나 모델링 목적으로 데이터셋을 깨끗하게 하기 위해서는 피처의 분포를 확인해야 하고 사전에 정의된 조건들을 만족하는지 검증 해야 한다.

- 데이터과학자는 80~90%의 시간을 데이터를 다루거나 피쳐에 익숙해지는데 쓰게 된다.

In [1]:
import findspark, pyspark
findspark.find()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('appName').setMaster('local[*]')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession(sc)

In [2]:
# sc.stop()

In [3]:
df = spark.createDataFrame([
    (1, 144.5, 5.9, 33, 'M'),
    (2, 167.2, 5.4, 45, 'M'),
    (3, 124.1, 5.2, 23, 'F'),
    (4, 144.5, 5.9, 33, 'M'),
    (5, 133.2, 5.7, 54, 'F'),
    (3, 124.1, 5.2, 23, 'F'),
    (5, 129.2, 5.3, 42, 'M'),
], ['id', 'weight', 'height', 'age', 'gender'])

In [4]:
print('count of rows: {0}'.format(df.count()))

count of rows: 7


In [5]:
print('Count of distinct rows : {0}'.format(df.distinct().count()))

Count of distinct rows : 6


In [6]:
[c for c in df.columns if c != 'id']

['weight', 'height', 'age', 'gender']

In [7]:
df.columns

['id', 'weight', 'height', 'age', 'gender']

In [8]:
df = df.dropDuplicates(subset=[c for c in df.columns if c != 'id'])
df.show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  5| 133.2|   5.7| 54|     F|
|  1| 144.5|   5.9| 33|     M|
|  2| 167.2|   5.4| 45|     M|
|  3| 124.1|   5.2| 23|     F|
|  5| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



In [9]:
import pyspark.sql.functions as fn

In [10]:
df.agg(fn.count('id').alias('count'), fn.countDistinct('id').alias('distinct')).show()

+-----+--------+
|count|distinct|
+-----+--------+
|    5|       4|
+-----+--------+



monotonically_increasing_id() <= 각 행에 고유한 값을 부연한다.
각각의 파티션에 800억개의 데이터가 있고 파티션의 수가 약 10억개 미만인 데이터들에 대해서는 
monotonically_increasing_id() 함수가 고유한 ID 값을 부여 해준다.

** 주의 ** 스파크 이전 버전에서 monotonically_increasing_id() 함수는 같은 데이터프레임에서 여러번 작업이 이뤄졌을때만 ID값이 바뀌었는데
스파크 2.0에서는 고정된 ID값을 부여한다.

In [11]:
df.withColumn('new_id', fn.monotonically_increasing_id()).show()

+---+------+------+---+------+-------------+
| id|weight|height|age|gender|       new_id|
+---+------+------+---+------+-------------+
|  5| 133.2|   5.7| 54|     F|  25769803776|
|  1| 144.5|   5.9| 33|     M| 171798691840|
|  2| 167.2|   5.4| 45|     M| 592705486848|
|  3| 124.1|   5.2| 23|     F|1236950581248|
|  5| 129.2|   5.3| 42|     M|1365799600128|
+---+------+------+---+------+-------------+



- Missing observations
- 미관찰 값들을 다룰 수 있는 가장 간단한 방법은 미관찰 값을 가지고있는 모든 데이터를 제거 하는 것이다.

- 단, 너무 많은 데이터를 제거하지 않도록 조심해야함.
- 데이터셋 전체에서 미관찰 값의 분포에 따라 데이터셋 전체의 사용 가능성에 큰 영향을 미칠 수 있다.
- 데이터를 제거한 후 아주 작은 데이터만 남거나, 데이터가 절반 이상으로 줄었들었다면 어떤 피처가 빈칸을 가장 많이 가지고 있는지 확인하고 그 피처를 제거하는 것이 더 좋다.
    - 데이터가 참/거짓으로 구분이되면 Missing라는 세번째 카테고리를 넣으면 된다.
    - 데이터가 이미 카테고리를 가지고 있다면 Missing 카테고리를 집어 넣으면 된다.
    - 순서 혹은 숫자 데이터를 가지고 있을 경우에는 평균, 중간값 또는 미리 정의된 다른 값으로 바꿀 수 있다. (ex: 데이터 분포에 따라 첫번째, 세번째 쿼타열 등 .....)

In [12]:
df_miss = spark.createDataFrame( [ 
    (1, 144.5, 5.6, 28, 'M', 100000),
    (2, 167.2, 5.4, 45, 'M', None),
    (3, None , 5.2, None, None, None),
    (4, 144.5, 5.9, 33, 'M', None),
    (5, 133.2, 5.7, 54, 'F', None),
    (6, 124.1, 5.2, None, 'F', None),
    (7, 129.2, 5.3, 42, 'M', 76000) ],
    ['id', 'weight', 'height', 'age', 'gender', 'income']
)


In [13]:
df_miss.rdd.map(
    lambda row : (row['id'], sum([c == None for c in row]))
).collect()

[(1, 0), (2, 1), (3, 4), (4, 1), (5, 1), (6, 2), (7, 0)]

In [14]:
df_miss.where('id==3').show()

+---+------+------+----+------+------+
| id|weight|height| age|gender|income|
+---+------+------+----+------+------+
|  3|  null|   5.2|null|  null|  null|
+---+------+------+----+------+------+



In [15]:
df_miss.agg(*[(1-(fn.count(c)/fn.count('*'))).alias(c + 'missing') for c in df_miss.columns]).show()

+---------+------------------+-------------+------------------+------------------+------------------+
|idmissing|     weightmissing|heightmissing|        agemissing|     gendermissing|     incomemissing|
+---------+------------------+-------------+------------------+------------------+------------------+
|      0.0|0.1428571428571429|          0.0|0.2857142857142857|0.1428571428571429|0.7142857142857143|
+---------+------------------+-------------+------------------+------------------+------------------+



In [16]:
df_miss_no_income = df_miss.select([c for c in df_miss.columns if c != 'income'])

In [17]:
df_miss.columns

['id', 'weight', 'height', 'age', 'gender', 'income']

In [18]:
df_miss_no_income.dropna(thresh=3).show()

+---+------+------+----+------+
| id|weight|height| age|gender|
+---+------+------+----+------+
|  1| 144.5|   5.6|  28|     M|
|  2| 167.2|   5.4|  45|     M|
|  4| 144.5|   5.9|  33|     M|
|  5| 133.2|   5.7|  54|     F|
|  6| 124.1|   5.2|null|     F|
|  7| 129.2|   5.3|  42|     M|
+---+------+------+----+------+



- 미관찰값을 추정해 채우려면 fillna() 함수를 사용 할 수 있다.
- 이 함수는 단일 integer, float, long, string 타입을 지원한다.
- 전체 데이터셋에서 미관찰값은 그 값으로 채워질 것이다.

- 평균, 중간값 또는 다른 계산된 값으로 채우려면 우선 그 값을 계산
- 그러한 값을 가지는 딕셔너리를 만든 후 fillna() 함수에 전달 하면 된다.

In [19]:
means = df_miss_no_income.agg(*[fn.mean(c).alias(c) for c in df_miss_no_income.columns if c != 'gender']).toPandas().to_dict('records')[0]

- toPandas() 함수는 collect()함수와 같은 방식으로 동작한다. 데이터 양에따라 문제가 발생 할 수 있다는 것을 기억해야 한다.
- toPandas() 함수는 모든 정보를 워커 노드로부터 수집한 후, 드라이버 노드로 옮긴다

In [20]:
means

{'id': 4.0, 'weight': 140.45, 'height': 5.471428571428571, 'age': 40.4}

In [21]:
df_miss_no_income.fillna(means).show()

+---+------+------+---+------+
| id|weight|height|age|gender|
+---+------+------+---+------+
|  1| 144.5|   5.6| 28|     M|
|  2| 167.2|   5.4| 45|     M|
|  3|140.45|   5.2| 40|  null|
|  4| 144.5|   5.9| 33|     M|
|  5| 133.2|   5.7| 54|     F|
|  6| 124.1|   5.2| 40|     F|
|  7| 129.2|   5.3| 42|     M|
+---+------+------+---+------+



# Outliers

In [22]:
df_outliers = spark.createDataFrame([
    (1, 144.5, 5.3, 33,),
    (2, 167.2, 5.5, 45,),
    (3, 244.1, 5.1, 99,),
    (4, 144.5, 5.5, 33,),
    (5, 133.2, 5.4, 54,),
    (6, 124.1, 5.1, 23,),
    (7, 129.2, 5.3, 42,),
], ['id', 'weight', 'height', 'age'])

- 아웃라이어는 대부분의 데이터와는 매우 다른 분포를 띄는 데이터를 말함
- 매우 다르다는 것에 대한 정의는 각자 다를 수 있다.

- ** 일반적으로 모든 값들이 대체적으로 Q1 - 1.5IQR 와 Q3 + 1.5IQR 사이에 있으면 아웃라이어가 없다고 말 할 수 있다.** 

- IQR은 상위 쿼타일(75%)과 하위 쿼타일(25%)의 차로 정의 된다.

- approxQuantile() 함수를 사용 할 수 있다.
- 첫번째 파라미터 : 컬럼명
- 두번째 파라미터 : 0과 1사이 값 (0.5 중간값)
- 세번째 파라미터 : 각 피처에 대한 허용 가능한 수준의 에러 (0으로 지정시 피처에 대한 정확한 값을 계산 할수 있으나 매우 많은 연산량을 요구한다


In [23]:
cols = ['weight', 'height', 'age']
bounds = {}

for col in cols:
    quantiles = df_outliers.approxQuantile(col, [0.25,0.75], 0.05)
    IQR = quantiles[1] - quantiles[0]
    bounds[col] = [quantiles[0] - 1.5 * IQR, quantiles[1] + 1.5 * IQR]

In [24]:
bounds

{'weight': [72.19999999999999, 224.2],
 'height': [4.499999999999999, 6.1000000000000005],
 'age': [1.5, 85.5]}

In [25]:
outliers = df_outliers.select(
    *['id'] + [
        (
            (df_outliers[c] < bounds[c][0]) | (df_outliers[c] > bounds[c][1])
        ).alias(c + '_o') for c in cols
    ]
)

In [26]:
outliers.show()

+---+--------+--------+-----+
| id|weight_o|height_o|age_o|
+---+--------+--------+-----+
|  1|   false|   false|false|
|  2|   false|   false|false|
|  3|    true|   false| true|
|  4|   false|   false|false|
|  5|   false|   false|false|
|  6|   false|   false|false|
|  7|   false|   false|false|
+---+--------+--------+-----+



In [27]:
df_outliers = df_outliers.join(outliers, on = 'id')

In [28]:
df_outliers.filter('age_o').select('id', 'age').show()

+---+---+
| id|age|
+---+---+
|  3| 99|
+---+---+



In [29]:
df_outliers.filter('weight_o').select('id', 'weight').show()

+---+------+
| id|weight|
+---+------+
|  3| 244.1|
+---+------+



# Understand data

- 기술통계
- 기술통계는 데이터셋에서의 관찰 값 개수, 각 컬럼의 평균, 표준 편차 또는 최댓값/최솟값 등의 기본적인 정보를 확인한다

In [30]:
import pyspark.sql.types as typ

- 이번 예제는 데이터가 커서 2개 cpu로는 부족
- \*로 해서 전체를 끌어서 사용했다.

In [31]:
fraud = sc.textFile('./data/ccFraud.csv.gz')

In [32]:
header = fraud.first()
header

'"custID","gender","state","cardholder","balance","numTrans","numIntlTrans","creditLine","fraudRisk"'

In [33]:
fraud = fraud\
        .filter(lambda row: row != header) \
        .map(lambda row: [int(elem) for elem in row.split(',')])

In [34]:
fields = [
    *[
        typ.StructField(h[1:-1], typ.IntegerType(),True)
        for h in header.split(',')
    ]
]

In [35]:
schema = typ.StructType(fields)

In [36]:
fraud_df = spark.createDataFrame(fraud, schema)

In [37]:
# 빈도수 구하기
fraud_df.groupby('gender').count().show()

+------+-------+
|gender|  count|
+------+-------+
|     1|6178231|
|     2|3821769|
+------+-------+



In [38]:
# describe 확인
# spark에서는 확인할 칼럼들을 적어주어야 한다.
numerical = ['balance', 'numTrans', 'numIntlTrans']

In [39]:
desc = fraud_df.describe(numerical)

In [40]:
desc.show()

+-------+-----------------+------------------+-----------------+
|summary|          balance|          numTrans|     numIntlTrans|
+-------+-----------------+------------------+-----------------+
|  count|         10000000|          10000000|         10000000|
|   mean|     4109.9199193|        28.9351871|        4.0471899|
| stddev|3996.847309737077|26.553781024522852|8.602970115863767|
|    min|                0|                 0|                0|
|    max|            41485|               100|               60|
+-------+-----------------+------------------+-----------------+



기술 통계는 작은 정보이지만 많은 것들을 알 수 있다. 

- 모든 피처는 양의 방향으로 왜곡이 됐다. (최댓값이 평균보다 몇 배 더 크다) 
- 변동 계수 coefficient variabioin (평균과 표준편차의 비율)가 매우 크다   
    (값이 1과 가깝거나 크다, 이는 넓게 퍼진 데이터를 의미함) 

In [41]:
#비대칭 확인 (balance 컬럼만 확인)
fraud_df.agg({'balance':'skewness'}).show()
# skewness -> 비대칭도
# https://ko.wikipedia.org/wiki/%EB%B9%84%EB%8C%80%EC%B9%AD%EB%8F%84

+------------------+
| skewness(balance)|
+------------------+
|1.1818315552995033|
+------------------+



편차 = X - M  
표준 편차 ~ (X - M)^2  
왜도 ~ (X - M)^3  
첨도 ~ (X - M)^4  

#### Correlations
corr() 함수는 두쌍의 상관계수만 계산할 수 있다. 피어슨 상관계수를 지원한다.

In [42]:

fraud_df.corr('balance', 'numTrans')

0.00044523140172659576

In [43]:
n_numerical = len(numerical)

In [44]:
corr = []
for i in range(0, n_numerical):
    temp = [None] * i
    
    for j in range(i, n_numerical):
        temp.append(fraud_df.corr(numerical[i], numerical[j]))
        corr.append(temp)

In [45]:
# numerical = ['balance', 'numTrans', 'numIntlTrans']
corr
# 대립가설에 대한 상관계수가 낮다
# => 쓸 수 있는 변수이다.

[[1.0, 0.00044523140172659576, 0.00027139913398184604],
 [1.0, 0.00044523140172659576, 0.00027139913398184604],
 [1.0, 0.00044523140172659576, 0.00027139913398184604],
 [None, 1.0, -0.0002805712819816179],
 [None, 1.0, -0.0002805712819816179],
 [None, None, 1.0]]

### Visualization
conda install -c conda-forge bkcharts  
conda install -c conda-forge holoviews


### Histogram
히스토그램은 피쳐의 분포를 시각화하는 가장 쉬운 방법이다.  
파이스파크에서 히스토그램을 만드는 방법은 세가지!  

- 데이터를 워커 노드에서 집계해서 워커 노드가 bin 리스트를 드라이버 노드에게 리턴하고 각 bin의 갯수를 드라이버 노드가 센다.
- 데이터를 모두 드라이버 노드에 리턴, 시각화 라이브러리 함수를 사용해서 히스토그램을 만든다.
- 데이터를 샘플링해서 드라이버 노드에 리턴, 드라이버 노드는 리턴된 데이터를 이용해 데이터를 시각화

데이터의 행 수가 너무 많으면 두번쨰 방법은 아예 불가능!!!! 